<a href="https://colab.research.google.com/github/Joan2022Laurente/fade/blob/main/notebooks/comfyui_colab_with_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# @title 🛠️ ComfyUI Centralized Setup (Optimizado para Azzia)
import os
from pathlib import Path

# --- CONFIGURACIÓN ---
HUB_NAME = "ComfyUI"  # Asegúrate de que el acceso directo en Drive se llame así
WORKSPACE = f"/content/drive/MyDrive/{HUB_NAME}"
UPDATE_SYSTEM = False  # @param {type:"boolean"}

# 1. Montaje de Drive
if not os.path.exists('/content/drive'):
    print("📂 Montando Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')

# 2. Verificación de Rutas y Acceso Directo
if not os.path.exists(WORKSPACE):
    print(f"❌ ERROR: No se encontró la carpeta en {WORKSPACE}")
    print("👉 SOLUCIÓN: Ve a 'Compartidos conmigo', busca la carpeta 'ComfyUI' y selecciona 'Añadir acceso directo a Drive'.")
else:
    print(f"✅ Hub detectado en: {WORKSPACE}")
    %cd {WORKSPACE}

# 3. Instalación de Motor Ultra-Rápido (uv) e Inserción de Dependencias
print("⚡ Instalando dependencias del sistema, Azzia y Nodos de Inpaint...")
!pip install uv -q
!uv pip install --system \
    pycryptodome requests pillow numpy av GitPython \
    accelerate einops transformers>=4.28.1 \
    safetensors>=0.4.2 aiohttp pyyaml \
    scipy tqdm psutil tokenizers>=0.13.3 \
    torchsde kornia>=0.7.1 spandrel soundfile \
    sentencepiece opencv-python \
    comfyui-workflow-templates comfyui-embedded-docs -q

# 4. Actualización Opcional (Solo si marcas el checkbox)
if UPDATE_SYSTEM:
    print("🔄 Sincronizando repositorios...")
    !git pull
    if os.path.exists("custom_nodes/ComfyUI-Manager"):
        %cd custom_nodes/ComfyUI-Manager
        !git pull
        %cd {WORKSPACE}

# 5. Finalización
print("\n✨ Entorno preparado.")
print("🚀 Tu nodo 'PostImageToAPI' (Azzia) ya tiene pycryptodome listo para cifrar.")
print("💡 Ya puedes ejecutar la celda para lanzar ComfyUI.")

✅ Hub detectado en: /content/drive/MyDrive/ComfyUI
/content/drive/MyDrive/ComfyUI
⚡ Instalando dependencias del sistema, Azzia y Nodos de Inpaint...

✨ Entorno preparado.
🚀 Tu nodo 'PostImageToAPI' (Azzia) ya tiene pycryptodome listo para cifrar.
💡 Ya puedes ejecutar la celda para lanzar ComfyUI.


### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
# @title ⚡ Launch ComfyUI
import subprocess, threading, time, socket

PORT = 8188
cf_deb = f"{WORKSPACE}/cloudflared-linux-amd64.deb"

# Instalación persistente de Cloudflared
if not os.path.exists(cf_deb):
    print("📥 Descargando binario de Cloudflare al Hub...")
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O {cf_deb}

!dpkg -i {cf_deb} > /dev/null 2>&1

def cloudflare_tunnel(port):
    while True:
        time.sleep(1)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        if sock.connect_ex(('127.0.0.1', port)) == 0:
            break
        sock.close()
    print("\n✨ Servidor listo. Generando enlace de acceso...\n")
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
                         stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print(f"🔗 URL DE ACCESO: {l[l.find('http'):]}")

threading.Thread(target=cloudflare_tunnel, daemon=True, args=(PORT,)).start()

# Ejecución limpia
!python main.py --dont-print-server --fast --preview-method auto

[START] Security scan
[ComfyUI-Manager] Using `uv` as Python module for pip operations.
Using Python 3.12.12 environment at: /usr
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-02-15 04:22:48.212
** Platform: Linux
** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/drive/MyDrive/ComfyUI
** ComfyUI Base Folder Path: /content/drive/MyDrive/ComfyUI
** User directory: /content/drive/MyDrive/ComfyUI/user
** ComfyUI-Manager config path: /content/drive/MyDrive/ComfyUI/user/__manager/config.ini
** Log path: /content/drive/MyDrive/ComfyUI/user/comfyui.log
Using Python 3.12.12 environment at: /usr
Using Python 3.12.12 environment at: /usr

Prestartup times for custom nodes:
   0.0 seconds: /content/drive/MyDrive/ComfyUI/custom_nodes/rgthree-comfy
   0.8 seconds: /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded